In [3]:
### RUN
# read from json file
import json 
with open('data.jsonl') as jsonl_file:
    lines = jsonl_file.readlines()
jsonS = [json.loads(line) for line in lines]
print(len(jsonS))

3702


In [4]:
### RUN
with open('annotations2.jsonl') as jsonl_file:
    # note: after running data-preprocessing.ipynb this file already has token-level labels
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]
#print(annot[0])

In [453]:
################ NO LONGER NECESSARY HERE AS RUN IN data-preprocessing.ipynb
def getLabel(listDict,idxEnd):
    # note: this function assumes annot[k]['tokens'] are laid out in ascending order in which they appear in text
    # this is not the case for the new 1 char tokens we added to correct labelling inconsistencies
    # but since they are all classifies as "O" anyway it doesn't matter 
    for i in range(0,len(listDict)):
        auxDict=listDict[i]
        a_s, a_e = auxDict['start'], auxDict['end']
        if idxEnd>=a_s: # i.e. in any case stop when idxStart < a_s (as dict tokens are ordered ASC in list; i.e. by order they appear in text)
            if (idxEnd>=a_s and idxEnd<=a_e): 
                return auxDict['label'] 
            elif i==len(listDict)-1: #case where idxStart > 'end' key of last dict in list
                if idxEnd>a_e:
                    return 'O'
        else: return 'O' 
# test function        
#a=[{'start':0, 'end':3, 'label': 'ORT'},{'start':5, 'end':8, 'label': 'ORT2'},{'start':10, 'end':13, 'label': 'ORT3'}]
#b=3
#print(getLabel(a,b))

Chose whether or not to add leading "B-" / "I-" to class labels

In [24]:
########## ONLY RUN IF WE WANT TO ADD LEADING "B-" / "I-" TO CLASS LABEL
# now use above code and loop through all items of annot list:
# addLeading=1 for "Yes" (i.e. add leading "B-","I-" to annot); 0 for "No" (i.e. add labels to annot simply as they are)
addLeading = 1

if addLeading == 1:
    for j in range(0,len(annot)):
        a = annot[j]
        # select list of dict of tokens w/ annnotations and add column w/ no. of words to each dict:
        b = a['spans']
        # add noWords to b dict. note: b is list of dicts w/ annotations; tokens not on this list don't have annotations
        if b!=[]: #i.e. only try to add annotations to tokens if there are annotations to begin with
            #print(b)
            for i in range(0,len(annot[j]['tokens'])):
                    # now break-up label into 1st occurrence (leading "B-") and subsequent occurrences (leading "I-") (only for non "O"'s)
                    if annot[j]['tokens'][i]['label'] != "O":
                        if i==0:
                            annot[j]['tokens'][i]['label'] = "B-" + annot[j]['tokens'][i]['label']
                        else: 
                            if annot[j]['tokens'][i]['label'] == annot[j]['tokens'][i-1]['label'][2:]: # need to remove the leading "B-" that we had already been added to c[i-1]
                                annot[j]['tokens'][i]['label'] = "I-" + annot[j]['tokens'][i]['label']
                            else:
                                annot[j]['tokens'][i]['label'] = "B-" + annot[j]['tokens'][i]['label'] 

In [34]:
### RUN
# now convert annotation tokens into list (sentences) of lists (tokens) format for sklearn_crfsuite.CRF
train_sents=[] 
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    train_sentence = []
    for i in range(0,len(a)):
        if 'label' in a[i]: # only add element if this sample sentence has been labelled 
            token_element = (a[i]['text'],a[i]['label'])
            train_sentence.append(token_element)
    train_sents.append(train_sentence)

In [35]:
print(train_sents[0])

[('DORNBIRN', 'B-ORT'), ('In', 'O'), ('der', 'O'), ('Schulgasse', 'B-STRASSE'), ('in', 'O'), ('Dornbirn', 'B-ORT'), ('hat', 'O'), ('eine', 'O'), ('71,93', 'B-FLAECHE'), ('Quadratmeter', 'O'), ('große', 'O'), ('Wohnung', 'B-IMMO_TYP'), ('für', 'O'), ('einen', 'O'), ('Quadratmeterpreis', 'O'), ('von', 'O'), ('5533,71', 'B-QMPREIS'), ('Euro', 'O'), ('den', 'O'), ('Besitzer', 'O'), ('gewechselt', 'O'), ('.', 'O'), ('Dieser', 'O'), ('beinhaltet', 'O'), ('auch', 'O'), ('einen', 'O'), ('Pkw-Abstellplatz', 'O'), ('.', 'O'), ('Käufer', 'O'), ('der', 'O'), ('Wohnung', 'O'), ('mit', 'O'), ('9,86', 'B-TERRASSENGROESSE'), ('Quadratmetern', 'O'), ('Terrasse', 'O'), ('ist', 'O'), ('die', 'O'), ('ValLiLean', 'B-KAEUFER'), ('Beteiligungs-', 'I-KAEUFER'), ('und', 'I-KAEUFER'), ('Immobilienverwaltungs', 'I-KAEUFER'), ('GmbH', 'I-KAEUFER'), ('.', 'O'), ('Beim', 'O'), ('Verkäufer', 'O'), ('handelt', 'O'), ('es', 'O'), ('sich', 'O'), ('um', 'O'), ('die', 'O'), ('Karrenblick', 'B-VERKAEUFER'), ('Projekt', 'I

In [425]:
print(annot[0]['spans'])

[{'text': 'DORNBIRN', 'start': 0, 'end': 8, 'pattern': 2069086582, 'token_start': 0, 'token_end': 0, 'label': 'ORT', 'noWords': 1}, {'start': 16, 'end': 26, 'token_start': 3, 'token_end': 3, 'label': 'STRASSE', 'noWords': 1}, {'text': 'Dornbirn', 'start': 30, 'end': 38, 'pattern': 2069086582, 'token_start': 5, 'token_end': 5, 'label': 'ORT', 'noWords': 1}, {'start': 48, 'end': 53, 'token_start': 8, 'token_end': 8, 'label': 'FLAECHE', 'noWords': 1}, {'start': 73, 'end': 80, 'token_start': 11, 'token_end': 11, 'label': 'IMMO_TYP', 'noWords': 1}, {'start': 113, 'end': 120, 'token_start': 16, 'token_end': 16, 'label': 'QMPREIS', 'noWords': 1}, {'start': 221, 'end': 225, 'token_start': 32, 'token_end': 32, 'label': 'TERRASSENGROESSE', 'noWords': 1}, {'start': 257, 'end': 311, 'token_start': 37, 'token_end': 41, 'label': 'KAEUFER', 'noWords': 5}, {'start': 351, 'end': 375, 'token_start': 49, 'token_end': 51, 'label': 'VERKAEUFER', 'noWords': 3}, {'text': '398.040', 'start': 402, 'end': 409, 

In [390]:
b=annot[0]['spans']
c=annot[0]['tokens']
i=74
print(b)
print(c[i]['start'])
print(getLabel(b,c[i]['end']))

[{'text': 'DORNBIRN', 'start': 0, 'end': 8, 'pattern': 2069086582, 'token_start': 0, 'token_end': 0, 'label': 'ORT', 'noWords': 1}, {'start': 16, 'end': 26, 'token_start': 3, 'token_end': 3, 'label': 'STRASSE', 'noWords': 1}, {'text': 'Dornbirn', 'start': 30, 'end': 38, 'pattern': 2069086582, 'token_start': 5, 'token_end': 5, 'label': 'ORT', 'noWords': 1}, {'start': 48, 'end': 53, 'token_start': 8, 'token_end': 8, 'label': 'FLAECHE', 'noWords': 1}, {'start': 73, 'end': 80, 'token_start': 11, 'token_end': 11, 'label': 'IMMO_TYP', 'noWords': 1}, {'start': 113, 'end': 120, 'token_start': 16, 'token_end': 16, 'label': 'QMPREIS', 'noWords': 1}, {'start': 221, 'end': 225, 'token_start': 32, 'token_end': 32, 'label': 'TERRASSENGROESSE', 'noWords': 1}, {'start': 257, 'end': 311, 'token_start': 37, 'token_end': 41, 'label': 'KAEUFER', 'noWords': 5}, {'start': 351, 'end': 375, 'token_start': 49, 'token_end': 51, 'label': 'VERKAEUFER', 'noWords': 3}, {'text': '398.040', 'start': 402, 'end': 409, 

In [404]:
print(annot[0]['text'][311:312])

.


In [14]:
### RUN
import re

In [100]:
aaa="350.000 Euro"
bbb="axyz"
regex = re.compile(r'(\d\.?)?(\d{3}\.?\d{3})? Euro')
matches = regex.finditer(aaa)
for match in matches:
    print(match)
print(bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',aaa)))

<re.Match object; span=(0, 12), match='350.000 Euro'>
True


In [6]:
### RUN
import pandas as pd
df = pd.read_csv('townsVoralberg.csv',names=['towns'])  

In [10]:
### RUN
x = lambda a: str(a).lower()

In [11]:
### RUN
df.head()
a = map(x,df['towns'])

In [15]:
### RUN
def word2features(sent, i):
    word = sent[i][0]
    word_l = word.lower()
    towns = map(x,df['towns'])
    #postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word_l,
        'word[-3:]': word[-3:], # 3 char ending of word (suffix)
        'word[-2:]': word[-2:], # 2 char ending of word (suffix)
        'word.isupper()': word.isupper(), # all letters are caps
        'word.istitle()': word.istitle(), # 1st letter is a capital letter
        'word.isdigit()': word.isdigit(), # is digit?
        'word.isPrice()': bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',word)),
        'word.isTown()': word_l in towns,
        #'postag': postag, # don't have PoS data
        #'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0] # enriches feature function w/ immediately preceding word
        word1_l = word1.lower()
        #postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1_l,
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.isPrice()': bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',word1)),
            '-1.word.isTown()': word1_l in towns,
            #'-1:postag': postag1,
            #'-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0] # ... as well as w/ immediate sucessor word 
        word1_l = word1.lower()
        #postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1_l,
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1word.isPrice()': bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',word1)),
            '+1.word.isTown()': word1_l in towns,
            #'+1:postag': postag1,
            #'+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
   # return [label for token, postag, label in sent]
    return [label for token, label in sent]

def sent2tokens(sent):
   # return [token for token, postag, label in sent]
    return [token for token, label in sent]

In [16]:
sent2features(train_sents[0])[2]

{'bias': 1.0,
 'word.lower()': 'der',
 'word[-3:]': 'der',
 'word[-2:]': 'er',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isPrice()': False,
 'word.isTown()': False,
 '-1:word.lower()': 'in',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:word.isPrice()': False,
 '-1.word.isTown()': False,
 '+1:word.lower()': 'schulgasse',
 '+1:word.istitle()': True,
 '+1:word.isupper()': False,
 '+1word.isPrice()': False,
 '+1.word.isTown()': False}

In [36]:
####### RUN THIS ONE
X_train = [sent2features(s) for s in train_sents] # both are 140 x # of tokens in each example
y_train = [sent2labels(s) for s in train_sents]

In [104]:
print(len(X_train[1]))
print(len(y_train[1]))

72
72


In [37]:
####### RUN THIS ONE
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats
import eli5
%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train[:120], y_train[:120])

Wall time: 0 ns


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [38]:
####### RUN THIS ONE
eli5.show_weights(crf, top=30)

From \ To,O,B-DATUM_VERBUECHERUNG,I-DATUM_VERBUECHERUNG,B-DATUM_VERTRAG,I-DATUM_VERTRAG,B-FLAECHE,I-FLAECHE,B-GESAMTPREIS,I-GESAMTPREIS,B-IMMO_TYP,I-IMMO_TYP,B-KAEUFER,I-KAEUFER,B-ORT,B-QMPREIS,I-QMPREIS,B-STRASSE,I-STRASSE,B-TERRASSENGROESSE,B-VERKAEUFER,I-VERKAEUFER
O,2.607,1.527,-1.767,1.589,-2.004,2.169,-0.875,1.539,-1.513,2.077,-3.359,1.643,-2.124,0.939,1.146,-0.674,1.792,-2.305,0.621,1.787,-2.101
B-DATUM_VERBUECHERUNG,-0.153,-0.637,4.076,0.0,-0.147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-DATUM_VERBUECHERUNG,-0.001,0.0,2.255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-DATUM_VERTRAG,-0.593,-0.133,-0.0,-0.546,5.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-DATUM_VERTRAG,0.476,0.0,-0.59,0.0,3.439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-FLAECHE,1.201,0.0,0.0,0.0,-0.31,0.0,1.548,0.0,0.0,-0.163,-0.018,0.0,-0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-FLAECHE,0.152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-GESAMTPREIS,0.765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-GESAMTPREIS,0.068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-IMMO_TYP,0.502,0.0,-0.091,0.0,0.0,-0.048,0.0,0.634,0.0,-0.098,2.84,0.0,-0.084,0.0,-0.078,0.0,0.258,-0.05,0.0,0.0,-0.051


In [39]:
####### RUN THIS ONE
labels = list(crf.classes_)
#labels.remove('O')
X_test = X_train[120:]
y_test = y_train[120:]

In [40]:
####### RUN THIS ONE
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\metrics\_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.9702827196096833

In [41]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                       precision    recall  f1-score   support

                    O      0.990     0.989     0.989       974
B-DATUM_VERBUECHERUNG      1.000     1.000     1.000        15
I-DATUM_VERBUECHERUNG      1.000     1.000     1.000        14
      B-DATUM_VERTRAG      0.933     0.933     0.933        15
      I-DATUM_VERTRAG      0.941     1.000     0.970        16
            B-FLAECHE      0.944     1.000     0.971        17
            I-FLAECHE      0.000     0.000     0.000         0
        B-GESAMTPREIS      1.000     1.000     1.000        13
        I-GESAMTPREIS      0.000     0.000     0.000         0
           B-IMMO_TYP      0.941     0.727     0.821        22
           I-IMMO_TYP      0.000     0.000     0.000         0
            B-KAEUFER      0.600     0.750     0.667        12
            I-KAEUFER      0.750     0.600     0.667        10
                B-ORT      1.000     0.971     0.985        34
            B-QMPREIS      1.000     1.000     1.000  

C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['O', 'B-DATUM_VERBUECHERUNG', 'I-DATUM_VERBUECHERUNG', 'B-DATUM_VERTRAG', 'I-DATUM_VERTRAG', 'B-FLAECHE', 'I-FLAECHE', 'B-GESAMTPREIS', 'I-GESAMTPREIS', 'B-IMMO_TYP', 'I-IMMO_TYP', 'B-KAEUFER', 'I-KAEUFER', 'B-ORT', 'B-QMPREIS', 'I-QMPREIS', 'B-STRASSE', 'I-STRASSE', 'B-TERRASSENGROESSE', 'B-VERKAEUFER', 'I-VERKAEUFER'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\metrics\_cl

Below we collect various types of summary statistics of our labelled dataset

In [167]:
from collections import Counter 
# summary counts
# first unfold y_train
y_train_all = [item for sublist in y_train[120:] for item in sublist]
y_train_all_count = Counter(y_train_all)
print(y_train_all_count)
print(len(y_train_all))
print(len(y_train))

Counter({'O': 920, 'B-ORT': 34, 'I-DATUM_VERTRAG': 28, 'B-IMMO_TYP': 22, 'I-VERKAEUFER': 22, 'I-KAEUFER': 18, 'B-FLAECHE': 17, 'B-VERKAEUFER': 15, 'B-DATUM_VERTRAG': 15, 'B-DATUM_VERBUECHERUNG': 15, 'I-DATUM_VERBUECHERUNG': 15, 'B-STRASSE': 14, 'B-GESAMTPREIS': 13, 'B-KAEUFER': 12, 'B-QMPREIS': 11, 'I-STRASSE': 5, 'B-TERRASSENGROESSE': 5, 'I-ORT': 1, 'I-IMMO_TYP': 1})
1183
140


In [483]:
# extract counters for all annot 
for l in range(0,len(labels)):
    out=[]
    for k in range(0,len(annot)):
        j=len(annot[k]['tokens'])
        for i in range(0,j):
            if annot[k]['spans']!=[]: # are there annot for this example?
                if annot[k]['tokens'][i]['label']==labels[l]:
                    out.append(annot[k]['tokens'][i]['text'].lower())
    out_count = Counter(out)
    print(out_count)

Counter({'dornbirn': 37, 'feldkirch': 25, 'hohenems': 15, 'bregenz': 12, 'götzis': 9, 'lustenau': 9, 'schruns': 8, 'rankweil': 8, 'bludenz': 7, 'höchst': 6, 'lauterach': 5, 'ludesch': 4, 'vandans': 4, 'bludesch': 4, 'hörbranz': 4, 'lochau': 4, 'frastanz': 4, 'schwarzach': 4, 'nüziders': 4, 'wolfurt': 4, 'satteins': 4, 'koblach': 3, 'schlins': 3, 'zwischenwasser': 3, 'brand': 3, 'nenzing': 3, 'lech': 2, 'göfis': 2, 'sulzberg': 2, 'klaus-weiler': 2, 'egg': 2, 'hohenweiler': 2, 'bartholomäberg': 2, 'mäder': 2, 'bürs': 2, 'riefensberg': 2, 'altach': 2, 'klösterle': 2, 'blons': 2, 'fontanella': 2, 'schröcken': 2, 'hard': 2, 'krumbach': 2, 'doren': 2, 'zürs': 1, 'gaissau': 1, 'gaißau': 1, 'bürserberg': 1, 'fußach': 1, 'klaus': 1})
Counter({'der': 6, 'im': 5, 'in': 4, 'walgaustraße': 3, 'radetzkystraße': 3, 'bereich': 2, 'bachgasse': 2, '9': 2, 'quellengasse': 2, '19': 2, '7': 2, 'auf': 2, 'römerstraße': 2, 'hag': 2, 'bergäcker': 2, 'sandgasse': 2, '2': 2, '35': 2, 'landstraße': 2, 'schulgass

In [484]:
print(labels)

['ORT', 'STRASSE', 'FLAECHE', 'IMMO_TYP', 'QMPREIS', 'TERRASSENGROESSE', 'KAEUFER', 'VERKAEUFER', 'GESAMTPREIS', 'DATUM_VERTRAG', 'DATUM_VERBUECHERUNG']


In [485]:
# extract # of tokens for each annot 
out=Counter({})
for k in range(0,len(annot)):
    j=len(annot[k]['tokens']) #74
    i=0
    if annot[k]['spans']!=[]: # are there annot for this example?
        while i < j:
            m=0 # count length of successive tokens w/ same label
            #print("k/"+str(k))
            #print("i/"+str(i))
            while (i < (j-1)) and annot[k]['tokens'][i]['label']==annot[k]['tokens'][i+1]['label']:
                 # note w/ i=j-2 in below expression can go up to i+1 = j-1 ... i.e. i can go up to 72
                m+=1
                i+=1
                #print(annot[k]['tokens'][i]['label'])
                #print(m)
            m+=1
            out += Counter({(annot[k]['tokens'][i]['label'],m): 1}) # add 1 entry for this pair of label and length
            #if (annot[k]['tokens'][i]['label'] == 'GESAMTPREIS') and (m>1): # investigating situations where # successive tokens GESAMTPREIS > 1
            #    print("k/"+str(k))
            #    print("i/"+str(i))
            i+=1
            #print(out)
                   # out.append(annot[k]['tokens'][i]['text'].lower())
print(out)

Counter({('O', 2): 318, ('O', 1): 267, ('ORT', 1): 238, ('O', 5): 154, ('IMMO_TYP', 1): 138, ('FLAECHE', 1): 137, ('O', 7): 137, ('O', 4): 103, ('DATUM_VERTRAG', 2): 97, ('GESAMTPREIS', 1): 96, ('DATUM_VERBUECHERUNG', 2): 96, ('QMPREIS', 1): 75, ('VERKAEUFER', 1): 55, ('STRASSE', 1): 53, ('O', 6): 51, ('O', 8): 47, ('O', 3): 43, ('STRASSE', 2): 39, ('KAEUFER', 1): 38, ('TERRASSENGROESSE', 1): 36, ('O', 13): 18, ('O', 12): 18, ('O', 9): 17, ('KAEUFER', 2): 17, ('O', 10): 16, ('IMMO_TYP', 4): 14, ('VERKAEUFER', 2): 13, ('O', 11): 12, ('VERKAEUFER', 3): 11, ('KAEUFER', 3): 11, ('O', 14): 10, ('O', 15): 9, ('STRASSE', 3): 9, ('DATUM_VERTRAG', 3): 9, ('DATUM_VERBUECHERUNG', 1): 8, ('VERKAEUFER', 5): 7, ('KAEUFER', 4): 7, ('KAEUFER', 5): 4, ('DATUM_VERBUECHERUNG', 3): 4, ('VERKAEUFER', 4): 4, ('VERKAEUFER', 8): 3, ('IMMO_TYP', 5): 3, ('O', 18): 3, ('VERKAEUFER', 6): 2, ('STRASSE', 4): 2, ('GESAMTPREIS', 2): 2, ('IMMO_TYP', 2): 2, ('VERKAEUFER', 7): 2, ('O', 16): 2, ('IMMO_TYP', 3): 2, ('O', 

In [487]:
# find situations where last token of same label ends with non-alphanumeric character
for l in range(0,len(labels)):
    out=[]
    for k in range(0,len(annot)):
        j=len(annot[k]['tokens'])
        for i in range(0,j-1):
            if annot[k]['spans']!=[]: # are there annot for this example?
                if annot[k]['tokens'][i]['label']==labels[l]:
                    if bool(re.match(r'\W',annot[k]['tokens'][i]['text'].lower()[-1:])) and annot[k]['tokens'][i]['label'] != annot[k]['tokens'][i+1]['label']:
                        out.append(annot[k]['tokens'][i]['text'].lower())
                        #print("k/"+str(k))
        # now add for i=j-1 (i.e. last token in example; by definition this is last token in label)
        if annot[k]['spans']!=[]: # are there annot for this example?
            if annot[k]['tokens'][j-1]['label']==labels[l]:
                if bool(re.match(r'\W',annot[k]['tokens'][j-1]['text'].lower()[-1:])):
                    out.append(annot[k]['tokens'][j-1]['text'].lower())
    out_count = Counter(out)
    print(out_count)

Counter()
Counter()
Counter()
Counter()
Counter()
Counter()
Counter()
Counter({'.': 1})
Counter()
Counter()
Counter()


In [318]:
word='abcd2'
bool(re.match(r'\W',word[-1:]))

False

In [585]:
# Statistics of dataset
sumExamples = len(annot)
sumTokens = 0
sumLabels = 0
sumLabelledWords = 0
labelWords =[]
for i in range (0,sumExamples):
    sumTokens+=len(annot[i]['tokens'])
    noLabels = len(annot[i]['spans'])
    sumLabels += noLabels
    for j in range(0,noLabels):
        labelWords.append((annot[i]['spans'][j]['label'],annot[i]['spans'][j]['noWords'])) 
        sumLabelledWords+=annot[i]['spans'][j]['noWords']
print("Total Examples = " + str(sumExamples))
print("Total Distinct Labels = " + str(sumLabels))
print("Total Distinct Labelled Tokens = " + str(sumLabelledWords))
print("Avg. # of words per example = "+str(sumTokens/sumExamples))
print("Avg. # of labelled words per example = "+str(sumLabelledWords/sumExamples))
print("Avg. # of labels per example = "+str(sumLabels/sumExamples))
print("Avg. # of words per label = "+str(sumLabelledWords/sumLabels))
labelWordDistrib = Counter(labelWords)
labelWordDistrib_items = labelWordDistrib.items()
labelWordDistrib_sorted = sorted(labelWordDistrib_items)
print(labelWordDistrib_sorted)
# calc average words per label for each different label type:
    

Total Examples = 140
Total Distinct Labels = 1283
Total Distinct Labelled Tokens = 1822
Avg. # of words per example = 55.121428571428574
Avg. # of labelled words per example = 13.014285714285714
Avg. # of labels per example = 9.164285714285715
Avg. # of words per label = 1.4201091192517536
[(('DATUM_VERBUECHERUNG', 1), 8), (('DATUM_VERBUECHERUNG', 2), 96), (('DATUM_VERBUECHERUNG', 3), 4), (('DATUM_VERTRAG', 1), 1), (('DATUM_VERTRAG', 2), 97), (('DATUM_VERTRAG', 3), 9), (('FLAECHE', 1), 138), (('GESAMTPREIS', 1), 97), (('GESAMTPREIS', 2), 3), (('IMMO_TYP', 1), 142), (('IMMO_TYP', 2), 2), (('IMMO_TYP', 3), 2), (('IMMO_TYP', 4), 14), (('IMMO_TYP', 5), 3), (('KAEUFER', 1), 69), (('KAEUFER', 2), 17), (('KAEUFER', 3), 11), (('KAEUFER', 4), 7), (('KAEUFER', 5), 4), (('ORT', 1), 238), (('QMPREIS', 1), 76), (('STRASSE', 1), 53), (('STRASSE', 2), 39), (('STRASSE', 3), 9), (('STRASSE', 4), 2), (('TERRASSENGROESSE', 1), 36), (('VERKAEUFER', 1), 64), (('VERKAEUFER', 2), 13), (('VERKAEUFER', 3), 11)

In [584]:
#Average number of words per label for each different label type
a = [((a1,a2),a3) for ((a1,a2),a3) in labelWordDistrib.items()]

keys = [a1 for ((a1,a2),a3) in a]

values1 = [a3 for ((a1,a2),a3) in a]

values2 = [a2*a3 for ((a1,a2),a3) in a]

d = {}
e = {}
for item in range(len(keys)):
    key = keys[item]
    value1 = values1[item]
    value2 = values2[item]
    if key in d:
        d[key] += value1
        e[key] += value2
    else:
        d[key] = value1
        e[key] = value2
f={}
for key in d:
    f[key]=e[key]/d[key]
print(f)

{'ORT': 1.0, 'STRASSE': 1.6116504854368932, 'FLAECHE': 1.0, 'IMMO_TYP': 1.3680981595092025, 'QMPREIS': 1.0, 'TERRASSENGROESSE': 1.0, 'KAEUFER': 1.7037037037037037, 'VERKAEUFER': 2.1132075471698113, 'GESAMTPREIS': 1.03, 'DATUM_VERTRAG': 2.074766355140187, 'DATUM_VERBUECHERUNG': 1.962962962962963}


In [594]:
# lexicon
a=[]
for i in range(0,len(annot)):
    for j in range(0,len(annot[i]['tokens'])):
        a.append(annot[i]['tokens'][j]['text'].lower())
b=set(a)
print(len(a)) # total # of words
print(len(b)) # total # of distinct words

7717
1152
